In [2]:
import tkinter as tk
from tkinter import scrolledtext
import time
import re
import random
import nltk
from tkinter import messagebox

class SimonSaysGame:
    def __init__(self, master, sentence):
        self.master = master
        self.sentence = sentence
        self.words_in_sentence = separate_words(sentence)
        self.random_words = self.generate_random_words(len(self.words_in_sentence))
        self.sequence = self.words_in_sentence + self.random_words
        random.shuffle(self.sequence)
        self.user_sequence = []

        self.create_widgets()

    def create_widgets(self):
        self.info_label = tk.Label(self.master, text="Simon Says: Click the words in order.")
        self.info_label.grid(row=0, column=0, columnspan=5)

        num_columns = 5
        row = 1
        col = 0
        self.word_buttons = []
        for word in self.sequence:
            button = tk.Button(self.master, text=word, command=lambda w=word: self.check_sequence(w), state=tk.DISABLED)
            button.grid(row=row, column=col, padx=5, pady=5, sticky='nsew')
            self.word_buttons.append(button)

            col += 1
            if col == num_columns:
                col = 0
                row += 1

        self.master.after(1000, self.enable_buttons)

        # Configure rows and columns to expand with window resizing
        for i in range(row + 1):
            self.master.grid_rowconfigure(i, weight=1)
        for j in range(num_columns):
            self.master.grid_columnconfigure(j, weight=1)

    def enable_buttons(self):
        for button in self.word_buttons:
            button.config(state=tk.NORMAL)

    def check_sequence(self, word):
        self.user_sequence.append(word)
        if self.user_sequence == self.words_in_sentence:
            messagebox.showinfo("Simon Says", "Correct! Well done.")
            self.master.after(1000, self.next_sentence)
        elif self.user_sequence == self.words_in_sentence[:len(self.user_sequence)]:
            pass  # still in progress
        else:
            messagebox.showinfo("Simon Says", "Incorrect! Game Over. Try again.")
            self.reset_game()

    def next_sentence(self):
        messagebox.showinfo("Simon Says", "Great job! Moving on to the next sentence.")
        self.master.destroy()

    def reset_game(self):
        messagebox.showinfo("Simon Says", "Game Over. Try again.")
        self.master.destroy()

    def generate_random_words(self, num_words):
        english_words = set(nltk.corpus.words.words())
        random_words = random.sample(english_words, num_words)
        return random_words

class SnakeGame:
    def __init__(self, root, target_word):
        self.root = root
        self.root.title("Snake Game")
        self.root.geometry("400x400")
        self.root.resizable(False, False)

        self.canvas = tk.Canvas(root, bg="black", width=400, height=400)
        self.canvas.pack()

        self.target_word = target_word
        self.current_word = [target_word[0]]  # Start with the first letter
        self.snake = [(100, 100)]  # Snake starts with one segment
        self.poison_list = []
        self.food = self.create_food()
        self.poison1 = self.create_poison()
        self.poison2 = self.create_poison()
        self.poison_list.append(self.poison1)
        self.poison_list.append(self.poison2)

        self.direction = "Right"
        self.new_direction = "Right"

        self.root.bind("<KeyPress>", self.on_key_press)

        self.update()

    def create_food(self):
        x = random.randint(1, 19) * 20
        y = random.randint(1, 19) * 20
        food_letter = self.get_next_letter()

        self.canvas.create_text(x + 10, y + 10, text=food_letter, fill="cyan", font=("Helvetica", 20), tags="food")
        return x, y
    
    def create_poison(self):
        x = random.randint(1, 19) * 20
        y = random.randint(1, 19) * 20
        poison_letter = chr(random.randint(ord('a'), ord('z')))
        self.poison_list.append((x, y))

        self.canvas.create_text(x + 10, y + 10, text=poison_letter, fill="yellow", font=("Helvetica", 20), tags="poison")
        return x, y

    def get_next_letter(self):
        index = len(self.current_word)
        if index < len(self.target_word):
            return self.target_word[index]
        else:
            return ""

    def move(self):
        x, y = self.snake[0]

        if self.new_direction == "Right" and not self.direction == "Left":
            x += 20
        elif self.new_direction == "Left" and not self.direction == "Right":
            x -= 20
        elif self.new_direction == "Up" and not self.direction == "Down":
            y -= 20
        elif self.new_direction == "Down" and not self.direction == "Up":
            y += 20

        self.direction = self.new_direction

        self.snake.insert(0, (x, y))

        food_x, food_y = self.food
        food_letter = self.get_next_letter()
        if x == food_x and y == food_y:
            if food_letter and food_letter == self.target_word[len(self.current_word)]:
                self.current_word.append(food_letter)
                self.canvas.delete("food")
                #self.canvas.delete("poison")
                self.food = self.create_food()
                self.poison1 = self.create_poison()
                self.poison2 = self.create_poison()
            else:
                self.game_over(win=False)
        elif (x, y) in self.poison_list:
            self.game_over(win=False)
        else:
            self.canvas.delete("snake")
            self.snake.pop()

        self.check_collision()

        for i, segment in enumerate(self.snake):
            x, y = segment
            letter = self.current_word[i]
            self.canvas.create_text(x + 10, y + 10, text=letter, fill="light green", font=("Helvetica", 20), tags="snake")

        if self.current_word == list(self.target_word):
            self.game_over()

    def check_collision(self):
        x, y = self.snake[0]
        if x < 0 or x >= 400 or y < 0 or y >= 400 or len(self.snake) != len(set(self.snake)):
            self.game_over(win=False)

    def game_over(self, win=True):
        if win:
            message = f"Congratulations! You spelled '{self.target_word}'"
        else:
            message = "Game Over! You lost."

        self.canvas.create_text(200, 200, text=message, fill="white", font=("Helvetica", 16), tags="game_over")
        # Stop further updates
        self.update = lambda: None

    def reset_game(self):
        self.current_word = [self.target_word[0]]
        self.snake = [(100, 100)]
        self.food = self.create_food()
        self.poison1 = self.create_poison()
        self.poison2 = self.create_poison()
        self.direction = "Right"
        self.new_direction = "Right"
        self.canvas.delete("snake")
        self.canvas.delete("food")
        self.canvas.delete("poison")
        self.canvas.delete("game_over")
        self.update()

    def update(self):
        self.move()
        self.root.after(150, self.update)

    def on_key_press(self, event):
        key = event.keysym
        if (key == "Right" and not self.direction == "Left") or \
           (key == "Left" and not self.direction == "Right") or \
           (key == "Up" and not self.direction == "Down") or \
           (key == "Down" and not self.direction == "Up"):
            self.new_direction = key

def separate_words(input_text):
    words = re.findall(r'\b\w+\b', input_text)
    return words

def separate_sentences(input_text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', input_text)
    return sentences

def process_input():
    input_text = input_text_widget.get("1.0", tk.END)

    if input_text.strip():
        sentences = separate_sentences(input_text)

        sentences_listbox.delete(0, tk.END)
        words_listbox.delete(0, tk.END)

        for sentence in sentences:
            sentences_listbox.insert(tk.END, sentence)

        if sentences:
            simon_says_button.config(state=tk.NORMAL)
            simon_says_button.config(command=lambda: start_simon_says_game(sentences))

def show_words_for_sentence(event):
    selected_index = sentences_listbox.curselection()
    if selected_index:
        selected_sentence = sentences_listbox.get(selected_index)
        selected_words = separate_words(selected_sentence)
        words_listbox.delete(0, tk.END)
        for word in selected_words:
            words_listbox.insert(tk.END, word)
        
        # Enable the Simon Says button for the selected sentence
        simon_says_button.config(state=tk.NORMAL)
        simon_says_button.config(command=lambda: start_simon_says_game([selected_sentence]))
        
        snake_button.config(state=tk.NORMAL)
        snake_button.config(command=lambda: start_snake_game(selected_words))

def start_simon_says_game(sentences):
    game_window = tk.Toplevel(root)
    game_window.title("Simon Says Game")
    sentence = random.choice(sentences)
    simon_says_game = SimonSaysGame(game_window, sentence)

def start_snake_game(words):
    selected_index = words_listbox.curselection()
    if selected_index:
        selected_word = words_listbox.get(selected_index)
        game_window = tk.Toplevel(root)
        game_window.title("Snake Game")
        snake_game = SnakeGame(game_window, selected_word)

# Create the main window
root = tk.Tk()
root.title("Text Processing")

# Create input text widget
input_text_widget = scrolledtext.ScrolledText(root, width=40, height=10, wrap=tk.WORD)
input_text_widget.pack(padx=10, pady=10)

# Create button to process input
process_button = tk.Button(root, text="Process Text", command=process_input)
process_button.pack(pady=5)

# Create listboxes to display results
sentences_listbox = tk.Listbox(root, width=40, height=5)
sentences_listbox.pack(padx=10, pady=5)
sentences_listbox.bind('<ButtonRelease-1>', show_words_for_sentence)  # Bind click event to show words

words_listbox = tk.Listbox(root, width=40, height=5)
words_listbox.pack(padx=10, pady=5)

# Create Simon Says button (initially disabled)
simon_says_button = tk.Button(root, text="Start Simon Says Game", state=tk.DISABLED)
simon_says_button.pack(pady=5)

snake_button = tk.Button(root, text="Start Snake Game", state=tk.DISABLED, command=start_snake_game)
snake_button.pack(pady=5)

# Start the Tkinter event loop
root.mainloop()
